In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","12g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.microsoft.sqlserver:mssql-jdbc:9.4.0.jre8") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/worker2/.ivy2/cache
The jars for the packages stored in: /home/worker2/.ivy2/jars
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e428b0d6-a60f-41f6-a75d-9e02bc1806c6;1.0
	confs: [default]
	found com.microsoft.sqlserver#mssql-jdbc;9.4.0.jre8 in central
:: resolution report :: resolve 363ms :: artifacts dl 14ms
	:: modules in use:
	com.microsoft.sqlserver#mssql-jdbc;9.4.0.jre8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-e428b0d6

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[ WARN] Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
[ WARN] Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
CONNECTOR_TYPE='com.microsoft.sqlserver.jdbc.SQLServerDriver'
SQL_USERNAME='tyeson'
SQL_PASSWORD='Canhtoan1509'
SQL_SERVERNAME='tyeson.database.windows.net'
SQL_DBNAME='DW'
CONNECTION_STRING_SOURCE='jdbc:sqlserver://tyeson.database.windows.net:1433;databaseName=DW;'
CONNECTION_STRING_TARGET='jdbc:sqlserver://tyeson.database.windows.net:1433;databaseName=DM_COURSE_COURSE;'

In [5]:
fact_course = spark.read.format('jdbc')\
    .option("url", CONNECTION_STRING_SOURCE)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','fact_course')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .load()

In [6]:
dim_competency = spark.read.format('jdbc')\
    .option("url", CONNECTION_STRING_SOURCE)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_competency')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .load()

In [26]:
from pyspark.sql.functions import col

# Alias the fact_course dataframe for the self-join
fc_source = fact_course.select('courseKey', col('competencyId').alias('requiredId'), 'is_require')

# Join the fact_course and dim_competency dataframes to get the name of the required competency
fc_required = fc_source.join(dim_competency, fc_source.requiredId == dim_competency.competencyId) \
                     .where(col('is_require') == 1) \
                     .select(col('courseKey').alias('course_source_key'), 
                             col('competencyName').alias('course_require_competency'))

In [19]:
fc_teach = fact_course.alias('fc_teach') 

In [30]:
# Join the fact_course and dim_competency dataframes again to get the name of the competency taught by the course
fc_teach = fact_course.select('courseKey', col('competencyId').alias('teachId'), 'is_require')
fc_teach = fc_teach.join(dim_competency, fc_teach.teachId == dim_competency.competencyId, 'inner') \
                     .where(col('is_require') == 0) \
                     .select(col('courseKey').alias('course_key'), 
                             col('competencyName').alias('course_teach_competency'), 
                             col('competencyId'))

In [37]:
# Join the two dataframes on the competencyId to get the final result
result = fc_required.join(fc_teach, col('course_require_competency') == col('course_teach_competency')).select('*')

In [39]:
result= result.select('course_source_key', col('course_key').alias('course_target_id')).orderBy('course_source_key')

In [47]:
result.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING_TARGET)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','course_to_course')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()